First fine-tune weights

CAN THEN TRY CHANGING MODEL HYPERPARAMETERS

CAN ALSO TRY CHANGING TRAINING HYPERPARAMETERS

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset, load_dataset
import torch


df = pd.read_csv('./data/labeled_comments_2020.csv')
print(df.head(5))

df = df.rename(columns={'sentiment': 'label'})

def map_labels(rating):
    if rating == "negative":
        return 0  # Negative
    elif rating == "neutral":
        return 1  # Neutral
    else:
        return 2
    
df['label'] = df['label'].apply(map_labels)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Albert Huang\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Albert Huang\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Albert Huang\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [2]:
train_df, test_df = train_test_split(df, test_size=0.2)
# train_df, val_df = train_test_split(train_df, test_size=0.2)

train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import tensorflow as tf

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

c:\Users\Albert Huang\anaconda3\Lib\site-packages\numpy\_typing\_scalars.py:12: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _BoolLike_co = Union[bool, np.bool]


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
model.summary()

In [5]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [6]:
batch_size = 16
training_args = TrainingArguments(
    "fine-tuned-sentiment-analysis",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

c:\Users\Albert Huang\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# import numpy as np
# from datasets import load_metric
# metric = load_metric('glue', 'stsb')
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)

C:\Users\Albert Huang\AppData\Local\Temp\ipykernel_11764\527915761.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'stsb')


In [ ]:

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

C:\Users\Albert Huang\AppData\Local\Temp\ipykernel_11764\1166498585.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()

  0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: Found dtype Long but expected Float

In [ ]:
trainer.evaluate()

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
import torch.nn as nn

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.classifier = nn.Linear(model.config.hidden_size, 3)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

c:\Users\Albert Huang\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Albert Huang\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  


vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [6]:
print(train_dataset[0])

{'label': 'positive', 'input_ids': tensor([  101, 10191,   112,   161, 43586, 10151, 12671, 10488, 10574, 16360,
        10390, 19503, 10487, 10955, 62235, 10421, 10114, 10103, 16758,   119,
          151, 11530,   112,   162, 21506, 10197, 10127,   143, 17676, 10114,
        18979, 12787, 91426, 10108, 10955, 62235, 10421,   117, 10502, 10197,
        14693, 10346,   119, 10855, 16332, 11531, 11555, 56653, 10127, 10110,
        26936, 35382, 10140,   119,   143, 65910, 16222, 10600, 21103, 11727,
        11008, 10574, 73296, 69448, 10390, 11060, 10110, 11312,   112,   146,
        10346, 27048, 10171, 11250, 27435, 10421, 36710, 11370,   119, 10502,
        16209, 10578, 10320, 12125, 14618, 54770, 10114, 10487, 40874, 11240,
        10110, 15033,   117, 11681,   117, 10110, 12319, 58059, 10107, 10323,
        10287,   117, 24372,   117, 10110, 10103, 52373, 10108, 55899,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [13]:
print(f"Classifier output size: {model.classifier.out_features}")
print(train_dataset['label'][:100]) 

Classifier output size: 3
[0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1, 2, 1, 0, 2, 2, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 1, 1, 2, 1, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 0, 2, 0, 2, 0]


In [15]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8)
for batch in train_dataloader:
    print(batch['input_ids'].shape)  # Should match (batch_size, max_length)
    print(batch['label'].shape)     # Should match (batch_size,)
    break

torch.Size([8, 512])
torch.Size([8])


In [17]:
inputs = tokenizer("Sample text", return_tensors="pt", truncation=True, max_length=512)
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([1, 3])


In [18]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    evaluation_strategy="epoch",     # evaluate at each epoch
    save_strategy="epoch",           # Save model at each epoch
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # Load best model after training
    metric_for_best_model="accuracy" # Metric to select the best model
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

eval_log = None

trainer.train()

eval_log = trainer.evaluate()


print(f'Evaluation Log: {eval_log}')

c:\Users\Albert Huang\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/40 [00:00<?, ?it/s]

RuntimeError: shape '[-1, 5]' is invalid for input of size 24

In [ ]:
trainer.train()

eval_result = trainer.evaluate()

In [ ]:
print(eval_result)